In [ ]:
#import libraries here
# Base
import numpy as np
import PIL
import os
from datetime import datetime
import pandas as pd


# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix

# Plots
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from IPython.display import Image
%matplotlib inline
graphviz.set_jupyter_format('jpg')

# Torch
import torch
import torchvision
from torchvision import datasets, transforms
from torchvision.transforms import v2
from torch.utils.data import DataLoader, random_split

In [ ]:
!mkdir data

In [ ]:
!unzip -qq data/IntelClassification.zip -d $SLURM_TMPDIR/

In [ ]:
Image(filename=os.path.expandvars('datasets\data\input_1_1_6.jpg'))